# Data Exploration

Here, we take an initial look at the data. Our goal is to predict whether or not a conversion will happen. Conversion prediction is a binary classification problem. We have two data files: one has the feature values that resulted in a conversion, and another has feature values that didn't result in a conversion. We want to know:

* How large is the dataset?
* How many data points do we have for each class?
* What features are there?
* What are the data types?
* Are there null values?
* What is the features' distribution?
* Are there outliers?

In [15]:
import pandas as pd
import seaborn as sns
sns.set_theme(context='notebook', style='whitegrid')
from helper_functions.descriptive_statistics import descriptive_statistics, generate_histograms, generate_html_report

In [16]:
# Load the data into pandas dataframe
conversion_df = pd.read_csv('../data/0_raw/Conversion_data.csv')
conversion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   `SITE              100000 non-null  object 
 1   AD_FORMAT          100000 non-null  object 
 2   BROWSER_NAME       99968 non-null   object 
 3   SUPPLY_VENDOR      100000 non-null  object 
 4   METRO              97611 non-null   float64
 5   OS_FAMILY_NAME     99990 non-null   object 
 6   USER_HOUR_OF_WEEK  99987 non-null   float64
dtypes: float64(2), object(5)
memory usage: 5.3+ MB


In [17]:
nonconversion_df = pd.read_csv('../data/0_raw/nonconversion_data.csv')
nonconversion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SITE               300000 non-null  object 
 1   AD_FORMAT          300000 non-null  object 
 2   BROWSER_NAME       299827 non-null  object 
 3   SUPPLY_VENDOR      300000 non-null  object 
 4   METRO              278670 non-null  float64
 5   OS_FAMILY_NAME     299956 non-null  object 
 6   USER_HOUR_OF_WEEK  299966 non-null  float64
dtypes: float64(2), object(5)
memory usage: 16.0+ MB


In [18]:
conversion_df.head()

,`SITE,AD_FORMAT,BROWSER_NAME,SUPPLY_VENDOR,METRO,OS_FAMILY_NAME,USER_HOUR_OF_WEEK
0,www.auctionzip.com,728x90,Chrome,google,619.0,Windows,63.0
1,www.coolmathgames.com,160x600,Chrome,pubmatic,544.0,OS X,155.0
2,www.equibase.com,9544x9544,Chrome,sharethrough,617.0,Windows,155.0
3,www.manua.ls,320x50,Chrome,Media.Net,501.0,Android,92.0
4,www.wunderground.com,640x360,Chrome,yieldmo,613.0,Windows,91.0


In [19]:
nonconversion_df.head()

,SITE,AD_FORMAT,BROWSER_NAME,SUPPLY_VENDOR,METRO,OS_FAMILY_NAME,USER_HOUR_OF_WEEK
0,www.the-sun.com,640x360,Chrome,yieldmo,652.0,Windows,63.0
1,com.pixel.art.coloring.color.number,320x50,WebView,google,597.0,Android,64.0
2,www.yahoo.com,640x360,Chrome,yieldmo,563.0,Windows,64.0
3,www.fox61.com,9544x9544,Chrome,taboola,623.0,Windows,63.0
4,news.yahoo.com,300x250,Chrome,pubmatic,618.0,Windows,63.0


The conversion and non-conversion data are of the same format (except for the `SITE vs SITE column name difference). The conversion data frame has 100K rows, whereas the non-conversion data frame has 300K rows. Both datasets have fields with null values.
There are two numeric-valued columns, and the rest are categorical.

In [20]:
# Let's combine the conversion with nonconversion data in one data frame for convinience of exploration.
# Make columns' names the same
conversion_df.columns = nonconversion_df.columns

all_data = pd.concat([conversion_df, nonconversion_df]).reset_index(drop=True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SITE               400000 non-null  object 
 1   AD_FORMAT          400000 non-null  object 
 2   BROWSER_NAME       399795 non-null  object 
 3   SUPPLY_VENDOR      400000 non-null  object 
 4   METRO              376281 non-null  float64
 5   OS_FAMILY_NAME     399946 non-null  object 
 6   USER_HOUR_OF_WEEK  399953 non-null  float64
dtypes: float64(2), object(5)
memory usage: 21.4+ MB


In [21]:
all_data[:4]

,SITE,AD_FORMAT,BROWSER_NAME,SUPPLY_VENDOR,METRO,OS_FAMILY_NAME,USER_HOUR_OF_WEEK
0,www.auctionzip.com,728x90,Chrome,google,619.0,Windows,63.0
1,www.coolmathgames.com,160x600,Chrome,pubmatic,544.0,OS X,155.0
2,www.equibase.com,9544x9544,Chrome,sharethrough,617.0,Windows,155.0
3,www.manua.ls,320x50,Chrome,Media.Net,501.0,Android,92.0


## Numerical features' descriptive statistics

In [22]:
# Generate statistics
descriptive_statistics(
    all_data, 
    numerical_columns=all_data.select_dtypes(include=['number']).columns.to_list()
    )['descriptive_statistics']

,Column,Type,Count,Mean,Std Dev,Min,25%,50% (Median),75%,Max,Skewness,Kurtosis
0,METRO,Numerical,376281,657.23,4235.29,0.0,526.0,602.0,669.0,554053.0,94.32,9068.57
1,USER_HOUR_OF_WEEK,Numerical,399953,87.54,43.84,0.0,62.0,77.0,122.0,167.0,0.16,-0.86


A kurtosis of 9.069 for METRO values indicates that the distribution has heavy tails and a sharp peak, with significant extreme outliers. The high skewness of 94 suggests that the right tail is longer, while the majority of the data is concentrated on the left. The notable difference between the 75th quartile and the maximum value (554,053 - 669 = 553,384) further confirms this observation.

The standard deviation is substantial compared to the mean: 4,235 versus 657. Additionally, the interquartile range is narrow in relation to the overall data range: (669 - 526 = 143) compared to 554053. All of these factors reinforce the presence of extreme outliers.

In contrast, a kurtosis of -0.9 for USER_HOUR_OF_WEEK data indicates lighter tails and a flatter peak compared to a normal distribution. Here, the standard deviation is smaller than the mean, suggesting reasonable variability. The mean is less than 10% larger than the median, which confirms a light skewness of 0.16. The data range is 167, while the interquartile range is relatively narrow at (122 - 62 = 60). This indicates that 50% of the data falls within a moderately wide range of 60 units, suggesting that the data is fairly spread out.

Even though the METRO and USER_HOUR_OF_WEEK are presented as numerical features, they are categorical.

METRO values are the area codes; the difference between those numbers has no meaning. Thus, the values that appear as outliers might not necessarily be the ones. It would be beneficial to convert those area codes into coordinates to determine the data points' geographical distribution. We will keep the METRO values as is but treat them as categorical.

The same goes for the USER_HOUR_OF_WEEK feature. The values here range from 0 to 167, representing a consecutive hour in a 7-day week. That means an hour 0 is adjacent to the hour 167 of the previous week. We will treat this feature as categorical but transform it into more meaningful features: day and hour of the day.

In [23]:
# Let's have a look at numerical features distribution.
# After running this cell, go to output_images folder to see the plots
generate_histograms(
    all_data,
    numerical_columns=all_data.select_dtypes(include=['number']).columns.to_list()
)

[('numerical', 'METRO', None, 'output_images/METRO_numerical.png'),
 ('numerical',
  'USER_HOUR_OF_WEEK',
  None,
  'output_images/USER_HOUR_OF_WEEK_numerical.png')]

## Categorical features' descriptive statistics

In [24]:
# Generate statistics
descriptive_statistics(
    all_data, 
    categorical_columns=all_data.select_dtypes(include=['object']).columns.to_list()
    )['descriptive_statistics']

,Column,Type,Count,Unique,Most Frequent,Counts of Most Frequent,Least Frequent,Counts of Least Frequent,Base 2 Entropy
0,SITE,Categorical,400000,20687,www.yahoo.com,35119,www.dailykillersudoku.com,1,9.90
1,AD_FORMAT,Categorical,400000,12,9544x9544,89034,120x600,7,2.85
2,BROWSER_NAME,Categorical,399795,10,Chrome,298370,Yandex,3,1.21
3,SUPPLY_VENDOR,Categorical,400000,88,google,65374,smartclip,1,4.27
4,OS_FAMILY_NAME,Categorical,399946,6,Windows,176917,Other,660,1.66


The SITE column has 20687 unique values making it an extremely high cardinality feature. In addition, it seems that there are might be a lot of SITE values that appear only once or a small amount of times. The base 2 entropy of 10 indicates that on average, we'll need 10 bits to encode this feature. All of these suggest that the SITE feature might not be beneficial for our purpose.

## Histograms for low cardinality categorical features

Let's have a look at bar plots to get a better sense of categorical features distributions.

In [25]:
# Let's have a look at categrical features distribution.
# After running this cell, go to output_images folder to see the plots
generate_histograms(
    all_data,
    categorical_columns=all_data.drop(columns=['SITE']).select_dtypes(include=['object']).columns.to_list()
)

[('categorical', 'AD_FORMAT', None, 'output_images/AD_FORMAT_categorical.png'),
 ('categorical',
  'BROWSER_NAME',
  None,
  'output_images/BROWSER_NAME_categorical.png'),
 ('categorical',
  'SUPPLY_VENDOR',
  None,
  'output_images/SUPPLY_VENDOR_categorical.png'),
 ('categorical',
  'OS_FAMILY_NAME',
  None,
  'output_images/OS_FAMILY_NAME_categorical.png')]

## Numerical features' statistics by category

In [26]:
# Generate statistics
descriptive_statistics(
    all_data,
    numerical_columns = ['METRO'],
    segment_by=['OS_FAMILY_NAME', 'BROWSER_NAME']
    )['descriptive_statistics']


Statistics for METRO segmented by ['OS_FAMILY_NAME', 'BROWSER_NAME']:
                                 Count    Mean  Std Dev    Min     25%  50% (Median)     75%       Max  Skewness  Kurtosis
Android_Chrome                109468.0  714.61  6395.35  500.0  523.00         577.0  659.00  554053.0     62.00   3896.68
Android_Edge                      35.0  644.77   122.06  504.0  530.50         624.0  759.00     862.0      0.32     -1.42
Android_Firefox                   97.0  608.34   107.24  501.0  515.00         563.0  650.00     862.0      0.85     -0.67
Android_Opera                     16.0  654.50   127.59  501.0  550.75         628.5  760.25     881.0      0.33     -1.29
Android_Other                     37.0  612.51   107.71  501.0  515.00         573.0  725.00     807.0      0.59     -1.19
Android_WebView                41819.0  658.99  3829.12  500.0  528.00         606.0  693.00  392004.0    102.12  10433.23
Linux_Chrome                    7486.0  612.15   101.87  500.0  527.

""


In [27]:
generate_histograms(
    all_data,
    numerical_columns = ['METRO'],
    segment_by= ['OS_FAMILY_NAME', 'BROWSER_NAME']
)

[('segmented',
  'METRO',
  'Android_Chrome',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_Chrome.png"),
 ('segmented',
  'METRO',
  'Android_Edge',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_Edge.png"),
 ('segmented',
  'METRO',
  'Android_Firefox',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_Firefox.png"),
 ('segmented',
  'METRO',
  'Android_Opera',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_Opera.png"),
 ('segmented',
  'METRO',
  'Android_Other',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_Other.png"),
 ('segmented',
  'METRO',
  'Android_WebView',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Android_WebView.png"),
 ('segmented',
  'METRO',
  'Linux_Chrome',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Linux_Chrome.png"),
 ('segmented',
  'METRO',
  'Linux_Firefox',
  "output_images/['OS_FAMILY_NAME', 'BROWSER_NAME']_METRO_Linux_Firefox.png"),
 (

## Report

Let's generate an HTML report of selected features.

In [29]:
# Define multiple segmented statistics functions
segmented_stats_func1 = lambda data: descriptive_statistics(
    data,
    numerical_columns=['METRO'],
    segment_by=['OS_FAMILY_NAME'],
    show_output=False
)['segmented_statistics']

segmented_stats_func2 = lambda data: descriptive_statistics(
    data,
    numerical_columns=['USER_HOUR_OF_WEEK'],
    segment_by=['OS_FAMILY_NAME', 'BROWSER_NAME'],
    show_output=False
)['segmented_statistics']

# Define multiple segmented histogram functions
segmented_histograms_func1 = lambda data, output_dir: generate_histograms(
    data,
    numerical_columns=['METRO'],
    segment_by=['OS_FAMILY_NAME'],
    output_dir=output_dir
)

#segmented_histograms_func2 = lambda data, output_dir: generate_histograms(
#    data,
#    numerical_columns=['USER_HOUR_OF_WEEK'],
#    segment_by=['OS_FAMILY_NAME', 'BROWSER_NAME'],
#    output_dir=output_dir
#)

# Generate HTML report
generate_html_report(
    all_data,
    numerical_stats_func=lambda data: descriptive_statistics(
        data,
        numerical_columns=data.select_dtypes(include=['number']).columns.to_list(),
        show_output=False
    )['descriptive_statistics'],
    categorical_stats_func=lambda data: descriptive_statistics(
        data,
        categorical_columns=data.select_dtypes(include=['object']).columns.to_list(),
        show_output=False
    )['descriptive_statistics'],
    segmented_stats_funcs=[segmented_stats_func1, segmented_stats_func2],
    numerical_histograms_func=lambda data, output_dir: generate_histograms(
        data,
        numerical_columns=data.select_dtypes(include=['number']).columns.to_list(),
        output_dir=output_dir
    ),
    categorical_histograms_func=lambda data, output_dir: generate_histograms(
        data,
        categorical_columns=data.drop(columns=['SITE']).select_dtypes(include=['object']).columns.to_list(),
        output_dir=output_dir
    ),
    segmented_histograms_funcs=[segmented_histograms_func1],
    output_dir="output_images",
    output_file="../report.html"
)

HTML report generated: ../report.html
